In [1]:
import torch
from torchtext.datasets import IMDB
from torch.utils.data import DataLoader

In [22]:
data_iter = IMDB(split='train')

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [23]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

toker = get_tokenizer("basic_english")
train_iter = IMDB(split='train')

def yield_tokens(data_iter):
    for _,text in data_iter:
        yield toker(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter),specials=['unk'])
vocab.set_default_index(vocab['unk'])

AttributeError: ignored

In [ ]:
vocab(toker("I like ice"))

[12, 44, 2065]

In [ ]:
text_pipeline = lambda x: vocab(toker(x))
label_pipeline = lambda x: int(x) - 1

In [ ]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [ ]:
train_iter = IMDB(split="train")
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [ ]:
num_class = len(set([label for (label,text) in train_iter]))

In [ ]:
num_class

2

In [ ]:
len(vocab)

100683

In [1]:
cd C:\Users\ACER\Desktop\New folder\CS224n\Lecture 5 RNN, GRU, LSTM\

C:\Users\ACER\Desktop\New folder\CS224n\Lecture 5 RNN, GRU, LSTM


In [9]:
from model.rnn import RNN_Node,Rnn_Layer
import numpy as np

In [9]:
sample  = np.random.rand(10,20,100) 


In [7]:

x = sample[:,10,:]

x.shape

(10, 100)

In [10]:
x = sample[:,10,:]
h = np.random.rand(10,20) 
params_dict = {} 
params_dict['Ws'] = {}
params_dict['Ws']['value'] = np.random.rand(20,30)
params_dict['Ws']['grad'] = np.random.rand(20,30)


params_dict['Whh'] = {}
params_dict['Whh']['value'] = np.random.rand(20,20)
params_dict['Whh']['grad'] = np.random.rand(20,20)

params_dict['Whx'] = {}
params_dict['Whx']['value'] = np.random.rand(100,20)
params_dict['Whx']['grad'] = np.random.rand(100,20)


params_dict['bx'] = {}
params_dict['bx']['value'] = np.random.rand(1,20)
params_dict['bx']['grad'] = np.random.rand(1,20)


params_dict['by'] = {}
params_dict['by']['value'] = np.random.rand(1,30)
params_dict['by']['grad'] = np.random.rand(1,30)



In [11]:
node = RNN_Node()

In [16]:
a,b = node.forward(x,h,params_dict)

In [17]:
a.shape , b.shape

((10, 30), (10, 20))

In [10]:
c,d = node.backward(a,b,params_dict)

In [11]:
c.shape,d.shape

((10, 100), (10, 20))

In [10]:
layer = Rnn_Layer(20,30)

In [11]:
sample = np.random.rand(10,20,100) ## B,seq_len,vocab_size 

In [12]:
a = layer.forward(sample)

In [14]:
a[:,-1,:].shape

(10, 30)

In [15]:
w = np.random.rand(30,2)

In [21]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def dsigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    pass 
def dsoftmax(x): 
    pass 


In [27]:
sigmoid(a[:,-1,:])

array([[0.50085759, 0.5021686 , 0.49690615, 0.50270693, 0.49989327,
        0.5030807 , 0.49527235, 0.49974995, 0.49284697, 0.50112394,
        0.49775661, 0.49296719, 0.49668552, 0.49996545, 0.49942813,
        0.50646777, 0.50399902, 0.5079631 , 0.50081284, 0.5030274 ,
        0.50141925, 0.50504343, 0.50223784, 0.50167429, 0.49779305,
        0.49979441, 0.50202531, 0.49446501, 0.4998731 , 0.49931426],
       [0.50178073, 0.50173533, 0.49769692, 0.50104125, 0.50006837,
        0.50311008, 0.496516  , 0.49972588, 0.49333686, 0.50023844,
        0.49829529, 0.49458402, 0.49789401, 0.49845664, 0.49977592,
        0.50765122, 0.50024959, 0.50959658, 0.50018725, 0.50289055,
        0.50137008, 0.50460903, 0.50258423, 0.49984613, 0.49740869,
        0.49770381, 0.50279755, 0.49588026, 0.49900247, 0.50035003],
       [0.50038983, 0.50352791, 0.49748017, 0.50288353, 0.50019415,
        0.50293302, 0.4959516 , 0.49926732, 0.493567  , 0.50060363,
        0.4984303 , 0.49518138, 0.49758274, 0.

In [8]:
sample = np.random.rand(10,20,30)
layer.backward(sample).shape

(10, 20, 100)

In [ ]:
def train(num_iterations: int):
    plot_iter = np.zeros((0))
    plot_loss = np.zeros((0)) 

    num_iter = 0
    start_pos = 0

    while num_iter < num_iterations: 
        if 